## Install Packages

In [93]:
# !pip install awswrangler
# !pip install tableone
# !pip install tqdm

In [99]:
#Set environment variables for your notebook
import os 
project_id = 'mimic-369422'
os.environ['GOOGLE_CLOUD_PROJECT'] = project_id

import pydata_google_auth
credentials = pydata_google_auth.get_user_credentials(
    ['https://www.googleapis.com/auth/bigquery'],use_local_webserver=False
)

from google.cloud import bigquery

bigquery_client = bigquery.Client()#Write Query on BQ

def run_query(query):
    return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      credentials=credentials,
      configuration={'query': {
          'useLegacySql': False
}})

/home/jupyter-zaiman/.conda/envs/hiti/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


## Import Packages

In [18]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from tableone import TableOne

In [19]:
# Load cohort created in the cohort notebook.
cohort = pd.read_csv('initial_cohort_no_covariates.csv')
cohort_pats = cohort['subject_id'].unique()

In [20]:
cohort.shape

(475839, 38)

In [21]:
cohort.head()

,subject_id,hadm_id,stay_id,charttime,creat_low_past_7day,creat_low_past_48hr,creat,aki_stage_creat,uo_rt_6hr,uo_rt_12hr,...,intime,outtime,los,gender,anchor_age,anchor_year,anchor_year_group,dod,creat_24,urine_24
0,17159168,20979477,32985479,2121-01-19 17:00:00,NaN,NaN,NaN,NaN,0.4820,0.7369,...,2121-01-18 04:21:27,2121-01-22 15:44:50,4.474572,M,57.0,2121,2008 - 2010,2121-08-22,1,1
1,17159168,20979477,32985479,2121-01-20 13:00:00,NaN,NaN,NaN,NaN,0.4959,0.8138,...,2121-01-18 04:21:27,2121-01-22 15:44:50,4.474572,M,57.0,2121,2008 - 2010,2121-08-22,1,1
2,17159168,20979477,32985479,2121-01-19 23:00:00,NaN,NaN,NaN,NaN,0.4401,0.4610,...,2121-01-18 04:21:27,2121-01-22 15:44:50,4.474572,M,57.0,2121,2008 - 2010,2121-08-22,1,1
3,17159168,20979477,32985479,2121-01-20 20:00:00,NaN,NaN,NaN,NaN,0.4540,0.4139,...,2121-01-18 04:21:27,2121-01-22 15:44:50,4.474572,M,57.0,2121,2008 - 2010,2121-08-22,1,1
4,17159168,20979477,32985479,2121-01-20 23:00:00,NaN,NaN,NaN,NaN,0.5414,0.4401,...,2121-01-18 04:21:27,2121-01-22 15:44:50,4.474572,M,57.0,2121,2008 - 2010,2121-08-22,1,1


In [22]:
list(cohort)

['subject_id',
 'hadm_id',
 'stay_id',
 'charttime',
 'creat_low_past_7day',
 'creat_low_past_48hr',
 'creat',
 'aki_stage_creat',
 'uo_rt_6hr',
 'uo_rt_12hr',
 'uo_rt_24hr',
 'aki_stage_uo',
 'aki_stage',
 'admittime',
 'dischtime',
 'deathtime',
 'admission_type',
 'admission_location',
 'discharge_location',
 'insurance',
 'language',
 'marital_status',
 'race',
 'edregtime',
 'edouttime',
 'hospital_expire_flag',
 'first_careunit',
 'last_careunit',
 'intime',
 'outtime',
 'los',
 'gender',
 'anchor_age',
 'anchor_year',
 'anchor_year_group',
 'dod',
 'creat_24',
 'urine_24']

In [23]:
cohort['charttime'] = pd.to_datetime(cohort['charttime'])
cohort['intime'] = pd.to_datetime(cohort['intime'])
cohort['outtime'] = pd.to_datetime(cohort['outtime'])

In [24]:
cohort_icu_48 = cohort[(cohort['charttime'] >= cohort['intime']) & (cohort['charttime'] <= (cohort['intime'] + np.timedelta64(2, 'D')))]

In [25]:
cohort_icu_48.shape

(225392, 38)

In [26]:
min_stay = cohort_icu_48.groupby('stay_id')['intime'].min()
cohort_icu_48_first_aki = cohort_icu_48.merge(min_stay)

In [29]:
labs_q = f"""SELECT * 
FROM (select l.*, d.label from `physionet-data.mimiciv_hosp.labevents` AS l
JOIN `physionet-data.mimiciv_hosp.d_labitems` d 
ON l.itemid = d.itemid
WHERE l.itemid
IN (51006, 50822, 50971, 50810, 51221, 50912, 50862, 50820, 50882, 50803)) WHERE subject_id IN {str(tuple(cohort_pats))}"""
labs = run_query(labs_q)

In [30]:
labs.shape

(5116722, 16)

In [31]:
# Drop outpatient labs with null hadm ids. 
# Iterate to avoid memory error
cohort_icu_stay = cohort_icu_48_first_aki[['hadm_id', 'intime', 'outtime']]

In [32]:
labs = labs.dropna(subset=['hadm_id'])

In [33]:
labs_merged = labs.merge(cohort_icu_48_first_aki, on=['hadm_id', 'subject_id'])

In [34]:
labs_merged['charttime'] = pd.to_datetime(labs_merged['charttime_x'])
labs_merged['intime'] = pd.to_datetime(labs_merged['intime'])
labs_merged['outtime'] = pd.to_datetime(labs_merged['outtime'])

In [35]:
labs_filt = labs_merged[(labs_merged['charttime'] >= labs_merged['intime']) & (labs_merged['charttime'] <= labs_merged['outtime'])]

In [52]:
labs_filt.shape

(16086252, 53)

In [53]:
list(labs_merged)

['labevent_id',
 'subject_id',
 'hadm_id',
 'specimen_id',
 'itemid',
 'charttime_x',
 'storetime',
 'value',
 'valuenum',
 'valueuom',
 'ref_range_lower',
 'ref_range_upper',
 'flag',
 'priority',
 'comments',
 'label',
 'stay_id',
 'charttime_y',
 'creat_low_past_7day',
 'creat_low_past_48hr',
 'creat',
 'aki_stage_creat',
 'uo_rt_6hr',
 'uo_rt_12hr',
 'uo_rt_24hr',
 'aki_stage_uo',
 'aki_stage',
 'admittime',
 'dischtime',
 'deathtime',
 'admission_type',
 'admission_location',
 'discharge_location',
 'insurance',
 'language',
 'marital_status',
 'race',
 'edregtime',
 'edouttime',
 'hospital_expire_flag',
 'first_careunit',
 'last_careunit',
 'intime',
 'outtime',
 'los',
 'gender',
 'anchor_age',
 'anchor_year',
 'anchor_year_group',
 'dod',
 'creat_24',
 'urine_24',
 'charttime']

In [54]:
lab_description = pd.pivot_table(labs, values=['valuenum'], index=['hadm_id'], columns=['label'], aggfunc=[np.nanmin, np.nanmax])

In [55]:
lab_description = lab_description.reset_index()

In [56]:
cols = ["_".join(x) for x in list(lab_description)]
cols[0] = 'hadm_id'
lab_description.columns = cols

In [57]:
cohort_icu_labs = cohort_icu_48_first_aki.merge(lab_description, on='hadm_id')

In [58]:
cohort_icu_labs.head()

,subject_id,hadm_id,stay_id,charttime,creat_low_past_7day,creat_low_past_48hr,creat,aki_stage_creat,uo_rt_6hr,uo_rt_12hr,...,nanmax_valuenum_Albumin,nanmax_valuenum_Bicarbonate,"nanmax_valuenum_Calculated Bicarbonate, Whole Blood",nanmax_valuenum_Creatinine,nanmax_valuenum_Hematocrit,"nanmax_valuenum_Hematocrit, Calculated",nanmax_valuenum_Potassium,"nanmax_valuenum_Potassium, Whole Blood",nanmax_valuenum_Urea Nitrogen,nanmax_valuenum_pH
0,17159168,20979477,32985479,2121-01-19 17:00:00,NaN,NaN,NaN,NaN,0.4820,0.7369,...,2.4,34.0,NaN,2.0,30.1,29.0,5.3,4.4,72.0,7.47
1,17159168,20979477,32985479,2121-01-19 23:00:00,NaN,NaN,NaN,NaN,0.4401,0.4610,...,2.4,34.0,NaN,2.0,30.1,29.0,5.3,4.4,72.0,7.47
2,17159168,20979477,32985479,2121-01-19 20:00:00,NaN,NaN,NaN,NaN,0.4610,0.5728,...,2.4,34.0,NaN,2.0,30.1,29.0,5.3,4.4,72.0,7.47
3,17159168,20979477,32985479,2121-01-19 18:00:00,NaN,NaN,NaN,NaN,0.4401,0.6601,...,2.4,34.0,NaN,2.0,30.1,29.0,5.3,4.4,72.0,7.47
4,17159168,20979477,32985479,2121-01-18 12:00:00,NaN,NaN,NaN,NaN,0.4890,0.5508,...,2.4,34.0,NaN,2.0,30.1,29.0,5.3,4.4,72.0,7.47


In [59]:
cohort_icu_labs.shape

(225340, 58)

In [60]:
sofa = run_query("""SELECT * FROM `physionet-data.mimiciv_derived.sofa`""")
antib = run_query(
    """SELECT a.stay_id, a.starttime, a.route, a.antibiotic FROM `physionet-data.mimiciv_derived.antibiotic` as a
where a.antibiotic like "Vancomycin" or a.antibiotic like "Gentamicin" or a.antibiotic like "Vancocin" or a.antibiotic like "Gentamicin (Bulk)" or a.antibiotic like 'NEO*IV*Gentamicin' or a.antibiotic like 'NEO*IV*Vancomycin' or a.antibiotic like 'Gentamicin Sulfate'
""")
sepsis = run_query("""SELECT * FROM `physionet-data.mimiciv_derived.sepsis3`""")

In [61]:
cohort_sepsis = cohort_icu_labs.merge(sepsis, on='stay_id', how='left')

In [62]:
cohort_sepsis['sepsis3'].value_counts(dropna=False)

True    150389
<NA>     74951
Name: sepsis3, dtype: Int64

In [63]:
cohort_sepsis['sepsis3'].fillna(False, inplace=True)

In [64]:
cohort_sepsis.shape

(225340, 71)

In [65]:
sofa_48 = sofa[sofa['hr'] <= 48]

In [66]:
sofa_48_min = dict(sofa_48.groupby(by='stay_id')['hr'].min())

In [67]:
sofa['min_hour'] = sofa['stay_id'].map(sofa_48_min)

In [68]:
sofa = sofa[sofa['min_hour'] == sofa['hr']]

In [69]:
sofa.shape

(76922, 30)

In [70]:
cohort_sofa = cohort_sepsis.merge(sofa, on='stay_id', how='left')

In [71]:
cohort_sofa.shape

(225340, 100)

In [72]:
cohort_sofa.nunique()

subject_id_x               23639
hadm_id                    23639
stay_id                    23639
charttime                 202968
creat_low_past_7day          134
                           ...  
cardiovascular_24hours         5
cns_24hours                    5
renal_24hours                  5
sofa_24hours                  17
min_hour                       1
Length: 100, dtype: int64

In [73]:
antib.head()

,stay_id,starttime,route,antibiotic
0,<NA>,2137-08-28 20:00:00,ORAL,Vancocin
1,<NA>,2151-10-18 17:00:00,ORAL,Vancocin
2,<NA>,2116-07-10 18:00:00,PO,Vancocin
3,<NA>,2151-10-25 20:00:00,PO,Vancocin
4,<NA>,2147-11-23 18:00:00,ORAL,Vancocin


In [74]:
antib['antibiotic'] = 1

In [75]:
cohort_stay_intime = cohort_sofa[['stay_id', 'intime']]

In [76]:
antib['starttime'] = pd.to_datetime(antib['starttime'])

In [79]:
antib = antib.dropna(subset='stay_id')

In [80]:
antib_intime_stayid = cohort_stay_intime.merge(antib, on='stay_id')

In [81]:
antib_intime_stayid['delta_days'] = (antib_intime_stayid['starttime'] - antib_intime_stayid['intime']) / np.timedelta64(1, 'D')

In [82]:
antib_intime_stayid_48 = antib_intime_stayid[antib_intime_stayid['delta_days'] <= 2]

In [83]:
antib_intime_stayid_48 = antib_intime_stayid_48.drop_duplicates(subset=['stay_id'])

In [84]:
cohort_antib = cohort_sofa.merge(antib_intime_stayid_48, on='stay_id', how='left')

In [85]:
cohort_antib['antibiotic'].fillna(0, inplace=True)

In [86]:
cohort_antib.shape

(225340, 105)

In [103]:
vitals_q = """
SELECT * FROM `physionet-data.mimiciv_derived.vitalsign`
"""
vitals = run_query(vitals_q)

In [104]:
vitals.shape

(10244724, 15)

In [105]:
vitals.head()

,subject_id,stay_id,charttime,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,temperature,temperature_site,spo2,glucose
0,13470745,33125931,2118-12-26 02:00:00,95.0,136.0,51.0,76.0,NaN,NaN,NaN,29.0,NaN,None,98.0,NaN
1,17355488,38398763,2152-04-12 21:00:00,99.0,93.0,51.0,59.0,93.0,51.0,59.0,29.0,36.94,Oral,97.0,NaN
2,10361825,32232273,2112-11-07 10:00:00,86.0,84.0,51.0,58.0,84.0,51.0,58.0,29.0,NaN,None,96.0,NaN
3,15032392,31662749,2182-10-25 14:00:00,91.0,107.0,51.0,68.0,NaN,NaN,NaN,29.0,NaN,None,98.0,NaN
4,15761754,36866282,2135-05-07 10:00:00,91.0,129.0,51.0,72.0,NaN,NaN,NaN,29.0,NaN,None,95.0,144.0


In [106]:
agg_cols = ['heart_rate', 'mbp', 'mbp_ni', 'resp_rate', 'spo2']
vitals_min = vitals.groupby('stay_id')[agg_cols].min().reset_index()
vitals_min.rename(dict(zip(agg_cols, ['min_' + x for x in agg_cols])), axis=1, inplace=True)
vitals_max = vitals.groupby('stay_id')[agg_cols].max().reset_index()
vitals_max.rename(dict(zip(agg_cols, ['max_' + x for x in agg_cols])), axis=1, inplace=True)

In [107]:
cohort_vitals = cohort_antib.merge(vitals_min, on='stay_id', how='left')
cohort_vitals = cohort_vitals.merge(vitals_max, on='stay_id', how='left')

In [108]:
cohort_vitals.nunique()

subject_id_x            23639
hadm_id                 23639
stay_id                 23639
charttime              202968
creat_low_past_7day       134
                        ...  
max_heart_rate            185
max_mbp                   353
max_mbp_ni                190
max_resp_rate              91
max_spo2                   19
Length: 115, dtype: int64

In [109]:
cohort_vitals = cohort_vitals.drop_duplicates()

In [110]:
cohort_vitals.shape

(225340, 115)

In [87]:
diagnosis = pd.read_csv('covariates.csv')
diagnosis = diagnosis.drop([x for x in list(diagnosis) if 'Unnamed' in x], axis=1)

In [111]:
cohort_diag = cohort_vitals.merge(diagnosis, on='stay_id', how='left')

In [112]:
cohort_diag.shape

(225340, 119)

In [113]:
cohort_diag.nunique()

subject_id_x            23639
hadm_id                 23639
stay_id                 23639
charttime              202968
creat_low_past_7day       134
                        ...  
max_spo2                   19
CKD                         2
DM                          2
Hypertension                2
Heart Failure               2
Length: 119, dtype: int64

In [114]:
cohort_diag.shape

(225340, 119)

In [115]:
list(cohort_diag)

['subject_id_x',
 'hadm_id',
 'stay_id',
 'charttime',
 'creat_low_past_7day',
 'creat_low_past_48hr',
 'creat',
 'aki_stage_creat',
 'uo_rt_6hr',
 'uo_rt_12hr',
 'uo_rt_24hr',
 'aki_stage_uo',
 'aki_stage',
 'admittime',
 'dischtime',
 'deathtime',
 'admission_type',
 'admission_location',
 'discharge_location',
 'insurance',
 'language',
 'marital_status',
 'race',
 'edregtime',
 'edouttime',
 'hospital_expire_flag',
 'first_careunit',
 'last_careunit',
 'intime_x',
 'outtime',
 'los',
 'gender',
 'anchor_age',
 'anchor_year',
 'anchor_year_group',
 'dod',
 'creat_24',
 'urine_24',
 'nanmin_valuenum_Albumin',
 'nanmin_valuenum_Bicarbonate',
 'nanmin_valuenum_Calculated Bicarbonate, Whole Blood',
 'nanmin_valuenum_Creatinine',
 'nanmin_valuenum_Hematocrit',
 'nanmin_valuenum_Hematocrit, Calculated',
 'nanmin_valuenum_Potassium',
 'nanmin_valuenum_Potassium, Whole Blood',
 'nanmin_valuenum_Urea Nitrogen',
 'nanmin_valuenum_pH',
 'nanmax_valuenum_Albumin',
 'nanmax_valuenum_Bicarbonate'

In [116]:
cohort_diag.to_csv('cohort_with_covariates.csv', index=False)

In [117]:
crrt = pd.read_csv('crrt_positive_cohort.csv')

In [118]:
list(crrt)

['stay_id',
 'charttime',
 'crrt_mode',
 'access_pressure',
 'blood_flow',
 'citrate',
 'current_goal',
 'dialysate_fluid',
 'dialysate_rate',
 'effluent_pressure',
 'filter_pressure',
 'heparin_concentration',
 'heparin_dose',
 'hourly_patient_fluid_removal',
 'prefilter_replacement_rate',
 'postfilter_replacement_rate',
 'replacement_fluid',
 'replacement_rate',
 'return_pressure',
 'ultrafiltrate_output',
 'system_active',
 'clots',
 'clots_increasing',
 'clotted',
 'subject_id',
 'hadm_id',
 'first_careunit',
 'last_careunit',
 'intime',
 'outtime',
 'los',
 'time_delta']

In [119]:
crrt['crrt'] = 1

In [120]:
crrt = crrt[['stay_id', 'crrt']]

In [121]:
cohort_diag_crrt = cohort_diag.merge(crrt, on='stay_id', how='left')

In [122]:
cohort_diag_crrt['crrt'].fillna(0, inplace=True)

In [123]:
columns = ['first_careunit', 'creat_low_past_48hr', 'aki_stage', 'race','min_heart_rate',
 'min_mbp',
 'min_mbp_ni',
 'min_resp_rate',
 'min_spo2',
 'max_heart_rate',
 'max_mbp',
 'max_mbp_ni',
 'max_resp_rate',
 'max_spo2',
 'CKD',
 'DM',
 'Hypertension',
 'Heart Failure',
'nanmin_valuenum_Albumin',
 'nanmin_valuenum_Bicarbonate',
 'nanmin_valuenum_Calculated Bicarbonate, Whole Blood',
 'nanmin_valuenum_Creatinine',
 'nanmin_valuenum_Hematocrit',
 'nanmin_valuenum_Hematocrit, Calculated',
 'nanmin_valuenum_Potassium',
 'nanmin_valuenum_Potassium, Whole Blood',
 'nanmin_valuenum_Urea Nitrogen',
 'nanmin_valuenum_pH',
 'nanmax_valuenum_Albumin',
 'nanmax_valuenum_Bicarbonate',
 'nanmax_valuenum_Calculated Bicarbonate, Whole Blood',
 'nanmax_valuenum_Creatinine',
 'nanmax_valuenum_Hematocrit',
 'nanmax_valuenum_Hematocrit, Calculated',
 'nanmax_valuenum_Potassium',
 'nanmax_valuenum_Potassium, Whole Blood',
 'nanmax_valuenum_Urea Nitrogen',
 'nanmax_valuenum_pH',
 'sepsis3',
 'sofa_24hours',
 'antibiotic',
 'gender']

numerical=[
    'creat_low_past_48hr',
    'min_heart_rate',
    'min_mbp',
 'min_mbp_ni',
 'min_resp_rate',
 'min_spo2',
 'max_heart_rate',
 'max_mbp',
 'max_mbp_ni',
 'max_resp_rate',
 'max_spo2','nanmin_valuenum_Albumin',
 'nanmin_valuenum_Bicarbonate',
 'nanmin_valuenum_Calculated Bicarbonate, Whole Blood',
 'nanmin_valuenum_Creatinine',
 'nanmin_valuenum_Hematocrit',
 'nanmin_valuenum_Hematocrit, Calculated',
 'nanmin_valuenum_Potassium',
 'nanmin_valuenum_Potassium, Whole Blood',
 'nanmin_valuenum_Urea Nitrogen',
 'nanmin_valuenum_pH',
 'nanmax_valuenum_Albumin',
 'nanmax_valuenum_Bicarbonate',
 'nanmax_valuenum_Calculated Bicarbonate, Whole Blood',
 'nanmax_valuenum_Creatinine',
 'nanmax_valuenum_Hematocrit',
 'nanmax_valuenum_Hematocrit, Calculated',
 'nanmax_valuenum_Potassium',
 'nanmax_valuenum_Potassium, Whole Blood',
 'nanmax_valuenum_Urea Nitrogen',
 'nanmax_valuenum_pH',
        'sofa_24hours',]

for col in numerical:
    cohort_diag_crrt[col] = cohort_diag_crrt[col].astype(np.float64)

categorical = [x for x in columns if x not in numerical]

tab = TableOne(cohort_diag_crrt, columns=columns,categorical=categorical, groupby='crrt')

/home/jupyter-zaiman/.local/lib/python3.9/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['percent'] = df['freq'].div(df.freq.sum(level=0),
/home/jupyter-zaiman/.local/lib/python3.9/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['percent'] = df['freq'].div(df.freq.sum(level=0),
/home/jupyter-zaiman/.local/lib/python3.9/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['percent'] = df['freq

In [114]:
tab.to_csv('tableone.csv')

In [124]:
cohort_diag_crrt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225340 entries, 0 to 225339
Columns: 120 entries, subject_id_x to crrt
dtypes: Int64(21), boolean(1), datetime64[ns](11), float64(61), int64(8), object(18)
memory usage: 219.3+ MB


In [125]:
tab.head(100)

Grouped by crrt  \
                                                                                 Missing   
n                                                                                          
first_careunit, n (%)   Cardiac Vascular Intensive Care Unit (CVICU)                   0   
                        Coronary Care Unit (CCU)                                           
                        Medical Intensive Care Unit (MICU)                                 
                        Medical/Surgical Intensive Care Unit (MICU/SICU)                   
...                                                                                  ...   
sofa_24hours, mean (SD)                                                                3   
antibiotic, n (%)       0.0                                                            0   
                        1.0                                                                
gender, n (%)           F                                                              0   
                        M                                                                  

                                                                                         \
                                                                                Overall   
n                                                                                225340   
first_careunit, n (%)   Cardiac Vascular Intensive Care Unit (CVICU)       55128 (24.5)   
                        Coronary Care Unit (CCU)                           22482 (10.0)   
                        Medical Intensive Care Unit (MICU)                 44021 (19.5)   
                        Medical/Surgical Intensive Care Unit (MICU/SICU)   29158 (12.9)   
...                                                                                 ...   
sofa_24hours, mean (SD)                                                       2.3 (2.4)   
antibiotic, n (%)       0.0                                               123575 (54.8)   
                        1.0                                               101765 (45.2)   
gender, n (%)           F                                                 101060 (44.8)   
                        M                                                 124280 (55.2)   

                                                                                         \
                                                                                    0.0   
n                                                                                212314   
first_careunit, n (%)   Cardiac Vascular Intensive Care Unit (CVICU)       52656 (24.8)   
                        Coronary Care Unit (CCU)                            20919 (9.9)   
                        Medical Intensive Care Unit (MICU)                 40066 (18.9)   
                        Medical/Surgical Intensive Care Unit (MICU/SICU)   27427 (12.9)   
...                                                                                 ...   
sofa_24hours, mean (SD)                                                       2.2 (2.3)   
antibiotic, n (%)       0.0                                               119603 (56.3)   
                        1.0                                                92711 (43.7)   
gender, n (%)           F                                                  96105 (45.3)   
                        M                                                 116209 (54.7)   

                                                                                       
                                                                                  1.0  
n                                                                               13026  
first_careunit, n (%)   Cardiac Vascular Intensive Care Unit (CVICU)      2472 (19.0)  
                        Coronary Care Unit (CCU)                          1563 (12.0)  
                        Medical Intensive Care Unit (MICU)          

In [126]:
cohort_diag_crrt.nunique()

subject_id_x            23639
hadm_id                 23639
stay_id                 23639
charttime              202968
creat_low_past_7day       134
                        ...  
CKD                         2
DM                          2
Hypertension                2
Heart Failure               2
crrt                        2
Length: 120, dtype: int64

In [ ]:
cohort_diag_crrt

In [132]:
first = cohort_diag_crrt.groupby('stay_id')['intime_x'].min()
cohort_diag_crrt = cohort_diag_crrt.merge(first)

In [133]:
cohort_diag_crrt.shape

(29093, 120)

In [129]:
cohort_diag_crrt.to_csv('final_cohort.csv', index=False)